In [ ]:
#Импортирование необходимых библиотек
import pandas as pd
import numpy as np
from pprint import pprint
import codecs
import json
import glob
pd.set_option('display.max_columns', None)
from pandas import json_normalize

In [ ]:
#Путь к файлам .geojson
path = 'Data'
file = glob.glob(path + "/*.json")

dataframes = []


#Цикл для получения файла и его загрузки, используя json.load 
df_data=[]
count = 0
for filename in file:
    name = filename.split("\\")[-1][:-5]
    
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
           
    for article in data['refs']:
        if article!=None:
            df_data.append([name, article[0], article[1]['day'], article[1]['month'], article[1]['time']])
            
df_full=pd.DataFrame(data=df_data,columns=['Название файла', 'Пост', 'день публикации', 'месяц публикации', 'время публикации'])

In [ ]:
df_full.head(5)

In [ ]:
#Путь к файлам .geojson
path = 'Data'
file = glob.glob(path + "/*.json")
df_full=pd.DataFrame()
names=[]
#df_full=pd.DataFrame(columns=['Пост', 'день публикации', 'месяц публикации', 'время публикации'])
#Цикл для получения файла и его загрузки, используя json.load 
for filename in file:
    name = filename.split("\\")[-1][:-5]
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
    
    for article in data['refs']:  
        if article != None:
            df=pd.concat(
                [
                    pd.DataFrame([article[0]],columns=['Post']),
                    json_normalize(article[1]),
                    pd.DataFrame([name],columns=['Company'])
                ],
                axis=1
            )
            df_full=pd.concat([df_full,df],axis=0,ignore_index=True)
        

In [ ]:
df_full.info()

In [ ]:
df_full.head()

Датафрейм с информацией о компании

In [ ]:
#Путь к файлам .geojson
path = 'Data'
file = glob.glob(path + "/*.json")
df_full=pd.DataFrame()
names=[]
#df_full=pd.DataFrame(columns=['Пост', 'день публикации', 'месяц публикации', 'время публикации'])
#Цикл для получения файла и его загрузки, используя json.load 
for filename in file:
    name = filename.split("\\")[-1][:-5]
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
    
    for article in data['refs']:  
        if article!= None:
            df=pd.concat(
                [
                    pd.DataFrame([article[0]],columns=['Post']),
                    json_normalize(article[1]),
                    pd.DataFrame([name],columns=['company'])
                ],
                axis=1
            )
            df_full=pd.concat([df_full,df],axis=0,ignore_index=True)
        

In [ ]:
#Путь к файлам .geojson
path = 'Data'
file = glob.glob(path + "/*.json")


df = pd.DataFrame(columns=['rate','subs','industries','about','company']) 

#Цикл для получения файла и его загрузки, используя json.load 
for filename in file:
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
        name = filename.split("\\")[-1][:-5]   
        try:
            company_info=pd.concat([json_normalize(data['info']),pd.DataFrame([name],columns=['company'])],axis=1)
        except:
            d={'rate':['Не указано'],'subs':['Не указано'],'industries':['Не указано'],'about':['Не указано']}
            company_info=pd.concat([pd.DataFrame(d),pd.DataFrame([name],columns=['company'])],axis=1)
    df = pd.concat([df,company_info], axis=0, ignore_index=True)
df.head()    
   

In [ ]:
tk = df_full.merge(df, on='company',how='left')

In [ ]:
tk.shape

In [ ]:
tk.head()

### Обработка текста

In [ ]:
import pymorphy2
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
sw = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

def clear_text(text):
    text=text.lower()
    text = re.sub(r'[^а-я ]','', str(text))
    tokens=word_tokenize(text, language="russian")
    tokens = [i for i in tokens if i not in sw and len(i) > 3]
    tokens = [morph.parse(i)[0].normal_form for i in tokens]
    return tokens

In [ ]:
tk['lemmatize_tokens'] = tk['Post'].apply(lambda x: clear_text(x))

In [ ]:
tk.head()

In [ ]:
tk['clear_text'] = tk['lemmatize_tokens'].apply(lambda x: " ".join(x))

In [ ]:
tk.head()

### Векторизация текста и поиск ngram

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [ ]:
tfidf = TfidfVectorizer(min_df=2,max_df=0.95,ngram_range=(1, 3))
X_tfidf = tfidf.fit_transform(tk['clear_text'])
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns = tfidf.get_feature_names())
df_tfidf.head()